In [1]:
 
import pandas as pd
import numpy as np

In [ ]:
cd gdrive/MyDrive/DM/Data/Beijing dataset/

In [6]:
hubs=[71808, 140297, 25230, 70940, 30, 175777, 172973, 70966, 310, 13497, 73665, 197828, 13514, 20682, 13516, 137676, 188878, 196301, 202, 231246, 83, 84, 20952, 172907, 159215, 117, 71928]

In [4]:
import pandas as pd
location = {}
#%cd 'gdrive/My Drive/Data Mining/Data/Beijing dataset'
nodeid=pd.read_csv('nodeid.csv')
for index, row in nodeid.iterrows():

    art = str(row['nodeid']).split('.')[0]
    cat = row['lon']
    cat2= row['lat']
    
    if art not in location.keys():
         location[art]=[]

    location[art].append(cat)
    location[art].append(cat2)

In [7]:
hubs_lon=[]
hubs_lat=[]
for id in hubs:
    hubs_lon.append(location[str(id)][0])
    hubs_lat.append(location[str(id)][1])

In [ ]:
cd Beijing_mapmatched_traj/

/content/gdrive/My Drive/DM/Data/Beijing dataset/Beijing_mapmatched_traj


In [8]:
import os
import pandas as pd

# Getting the current work directory (cwd)
thisdir = 'Beijing_mapmatched_traj/'


In [9]:
# r=root, d=directories, f = files
taxi = []
lis=[]
for r, d, f in os.walk(thisdir):
    for file in f:
        if file.endswith(".txt"):
            file = open(os.path.join(r, file),'r')
            traj_data = file.readlines()
            del traj_data[1::2]
            for ele in traj_data:
              if(ele!='broken\n'):
                #print(ele)
                try:
                  tmp = ele.split(': ')
                  tmp_ = tmp[1].split(' ')
                  t1=location[(tmp_[-2][1:-1])][0]
                  t2=location[(tmp_[-2][1:-1])][1]
                  t3=location[(tmp_[-1][:-2])][0]
                  t4=location[(tmp_[-1][:-2])][1]
                  taxi.append(t1)
                  taxi.append(t2)
                  taxi.append(t3)
                  taxi.append(t4) 
                  taxi.append(tmp[0][4:])
                except:
                  lis.append(tmp_)
len(lis)

1000

In [10]:
tmp_lis = np.reshape(taxi,(-1,5))

In [11]:
sorted_taxi = sorted(tmp_lis, key=lambda x: x[4])

In [12]:

node_id = nodeid['nodeid']


In [13]:
origin = []
dest = []
time_st = []
from random import randrange
import random
import time


start_timestamp = time.mktime(time.strptime('Jul 7 2008  01:00:00', '%b %d %Y %I:%M:%S'))
end_timestamp = time.mktime(time.strptime('Jul 8 2008  9:00:00', '%b %d %Y %I:%M:%S'))

def randomize_time(start_timestamp,end_timestamp):
    return time.strftime('%b %d %Y %I:%M:%S', time.localtime(randrange(start_timestamp,end_timestamp)))
for i in range(10):
  time_st.append(randomize_time(start_timestamp,end_timestamp))
  origin.append(str(node_id[random.randint(0,len(node_id))]))
  dest.append(str(node_id[random.randint(0,len(node_id))]))

In [14]:
from math import sin, cos, sqrt, atan2, radians

def Haversine(lat1,lon1,lat2,lon2, **kwarg):
    """
    This uses the ‘haversine’ formula to calculate the great-circle distance between two points – that is, 
    the shortest distance over the earth’s surface – giving an ‘as-the-crow-flies’ distance between the points 
    (ignoring any hills they fly over, of course!).
    Haversine
    formula:    a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
    c = 2 ⋅ atan2( √a, √(1−a) )
    d = R ⋅ c
    where   φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);
    note that angles need to be in radians to pass to trig functions!
    """
    R = 6371.0088
    lat1,lon1,lat2,lon2 = map(np.radians, [lat1,lon1,lat2,lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2) **2
    c = 2 * np.arctan2(a**0.5, (1-a)**0.5)
    d = R * c
    return round(d,4)

In [15]:
time_st_=[]
for i in time_st:
  time_st_.append(i.replace('2008 ',''))
time_st=time_st_

In [16]:
time_st[0]

'Jul 07 02:29:42'

In [17]:
sorted_taxi[-1]

array(['116.48395079999999', '39.9512713', '115.45068059999998',
       '39.7773093', 'Jul 08 09:31:23 IST 2013'], dtype='<U32')

In [19]:
dis_li =[]
dis = []
for i in range(1):
    pack_src_lon = location[origin[i][0:len(origin[i])-2]][0]
    pack_src_lat = location[origin[i][0:len(origin[i])-2]][1]
    pack_dest_lon = location[dest[i][0:len(dest[i])-2]][0]
    pack_dest_lat = location[dest[i][0:len(dest[i])-2]][1]
    pack_time = time_st[i]
    dis = list()
    dis_ = list()
    li=[]
    for j in range(len(sorted_taxi)):
      if sorted_taxi[j][4] > pack_time:
        li.append(sorted_taxi[j])
    for j in range(len(li)):
      dis.append(Haversine(pack_src_lat,pack_src_lon,float(li[j][0]),float(li[j][1])))
      x = np.array(hubs_lon)
      squarer = lambda t: t ** 2
      vfunc = np.vectorize(squarer)
      vfunc(x)
      [dis_.append(Haversine(float(li[i][0]),float(li[i][1]),hubs_lat[id],hubs_lon[id])) for id in range(len(hubs_lon))]
      hub_lon = hubs_lat[dis_.index(min(dis_))]
      hub_lat = hubs_lon[dis_.index(min(dis_))]
#     dis.append(Haversine(pack_dest_lat,pack_dest_lon,hub_lat,hub_lat))
# dis_li = np.reshape(dis,(-1,2))
# sorted_dis = sorted(dis_li, key=lambda x: x[0])

dis

KeyboardInterrupt: 

In [ ]:
origin[i][0:len(origin[i])-2]

In [ ]:
dis_li